# Aircraft Engine Monitoring - Data Exploration

This notebook explores the NASA C-MAPSS aircraft engine sensor data and demonstrates our real-time simulation capabilities.

## Overview

- **Objective**: Understand sensor patterns and degradation behaviors
- **Data Source**: NASA C-MAPSS dataset patterns
- **Focus**: 21 sensors covering temperature, pressure, speed, and flow metrics
- **Goal**: Build foundation for predictive maintenance analytics

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path to import our modules
sys.path.append('../src')

# Import our custom modules
from sensor_schema import EngineSchema
from data_simulator import EngineSimulator, FleetSimulator

# Set styling
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Explore Engine Schema and Sensor Configuration

In [ ]:
# Initialize the engine schema
schema = EngineSchema()

print("🔧 Aircraft Engine Sensor Configuration")
print("=" * 50)

# Create a summary DataFrame
sensor_info = []
for sensor_id, config in schema.sensors.items():
    sensor_info.append({
        'Sensor_ID': sensor_id,
        'Name': config.name,
        'Unit': config.unit,
        'Min_Range': config.normal_range[0],
        'Max_Range': config.normal_range[1],
        'Degradation_Pattern': config.degradation_pattern,
        'Noise_Level': config.noise_level,
        'Critical_Threshold': config.critical_threshold
    })

sensor_df = pd.DataFrame(sensor_info)
print(f"📊 Total Sensors: {len(sensor_df)}")
print(f"🔥 Temperature Sensors: {len(sensor_df[sensor_df['Unit'] == '°R'])}")
print(f"📈 Pressure Sensors: {len(sensor_df[sensor_df['Unit'] == 'psia'])}")
print(f"⚡ Speed Sensors: {len(sensor_df[sensor_df['Unit'] == 'rpm'])}")
print(f"⚠️  Degrading Sensors: {len(schema.get_degrading_sensors())}")
print(f"🚨 Critical Sensors: {len(schema.get_critical_sensors())}")

display(sensor_df)

## 3. Analyze Degradation Patterns

In [ ]:
# Analyze degradation patterns
degradation_analysis = sensor_df.groupby('Degradation_Pattern').agg({
    'Sensor_ID': 'count',
    'Noise_Level': 'mean',
    'Critical_Threshold': 'count'
}).round(2)

degradation_analysis.columns = ['Sensor_Count', 'Avg_Noise_Level', 'Total_Sensors']

print("📈 Degradation Pattern Analysis")
display(degradation_analysis)

# Visualize degradation patterns
fig = px.pie(sensor_df, names='Degradation_Pattern', 
             title='Distribution of Sensor Degradation Patterns',
             color_discrete_sequence=px.colors.qualitative.Set3)
fig.show()

print("\n🔍 Key Insights:")
print("• Temperature sensors (T24, T30, T50) show increasing degradation patterns")
print("• Speed sensors exhibit varying patterns as engines degrade")
print("• Pressure sensors remain relatively stable")
print("• Bypass ratio is a critical indicator of engine health")

## 4. Generate Simulated Engine Data

In [ ]:
# Create engines with different health states
engines = {
    'healthy': EngineSimulator("RR_TRENT_1001", schema, "healthy"),
    'degrading': EngineSimulator("RR_TRENT_1002", schema, "degrading"),
    'critical': EngineSimulator("RR_TRENT_1003", schema, "critical")
}

# Generate data for multiple cycles
simulation_data = []
cycles_to_simulate = 50

print(f"🔄 Generating {cycles_to_simulate} cycles of data for each engine...")

for cycle in range(1, cycles_to_simulate + 1):
    for health_state, engine in engines.items():
        reading = engine.generate_reading()
        
        # Flatten the data for analysis
        flat_data = {
            'cycle': reading['cycle'],
            'engine_id': reading['engine_id'],
            'health_state': reading['health_state'],
            'remaining_useful_life': reading['remaining_useful_life'],
            'alert_count': len(reading['alerts'])
        }
        
        # Add sensor values
        for sensor_id, value in reading['sensors'].items():
            flat_data[sensor_id] = value
        
        simulation_data.append(flat_data)

# Create DataFrame
df = pd.DataFrame(simulation_data)

print(f"✅ Generated {len(df)} data points")
print(f"📊 Data shape: {df.shape}")
print(f"🏥 Health states: {df['health_state'].value_counts().to_dict()}")

df.head()

## 5. Visualize Sensor Trends by Health State

In [ ]:
# Select key sensors for visualization
key_sensors = ['sensor_2', 'sensor_3', 'sensor_4', 'sensor_11']  # T24, T30, T50, Bypass_Ratio
sensor_names = {
    'sensor_2': 'Total Temperature T24 (°R)',
    'sensor_3': 'Total Temperature T30 (°R)', 
    'sensor_4': 'Total Temperature T50 (°R)',
    'sensor_11': 'Bypass Ratio'
}

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[sensor_names[sensor] for sensor in key_sensors],
    vertical_spacing=0.08
)

colors = {'healthy': 'green', 'degrading': 'orange', 'critical': 'red'}

for i, sensor in enumerate(key_sensors):
    row = (i // 2) + 1
    col = (i % 2) + 1
    
    for health_state in ['healthy', 'degrading', 'critical']:
        data = df[df['health_state'] == health_state]
        
        fig.add_trace(
            go.Scatter(
                x=data['cycle'],
                y=data[sensor],
                mode='lines+markers',
                name=f'{health_state.capitalize()}',
                line=dict(color=colors[health_state]),
                showlegend=(i == 0)  # Only show legend for first subplot
            ),
            row=row, col=col
        )

fig.update_layout(
    title="Key Sensor Trends by Engine Health State",
    height=600,
    showlegend=True
)

fig.show()

print("🔍 Observations:")
print("• Temperature sensors show clear increasing trends in degrading/critical engines")
print("• Bypass ratio increases significantly with engine degradation")
print("• Critical engines show more erratic sensor behavior")
print("• Healthy engines maintain stable sensor values with minimal drift")

## 6. Health State Classification Analysis

In [ ]:
# Analyze sensor statistics by health state
health_stats = df.groupby('health_state')[key_sensors].agg(['mean', 'std', 'min', 'max']).round(2)

print("📊 Sensor Statistics by Health State")
print("=" * 60)

for sensor in key_sensors:
    print(f"\n🔧 {sensor_names[sensor]}:")
    display(health_stats[sensor])

# Correlation analysis
print("\n🔗 Sensor Correlation Matrix")
correlation_matrix = df[key_sensors].corr()

# Create heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            square=True,
            xticklabels=[sensor_names[s] for s in key_sensors],
            yticklabels=[sensor_names[s] for s in key_sensors])
plt.title('Sensor Correlation Matrix')
plt.tight_layout()
plt.show()

print("\n🔍 Correlation Insights:")
print("• Temperature sensors (T24, T30, T50) are highly correlated")
print("• Bypass ratio shows inverse correlation with some temperature sensors")
print("• Strong correlations suggest redundancy in some measurements")

## 7. Alert Analysis

In [ ]:
# Analyze alert patterns
alert_analysis = df.groupby(['health_state', 'cycle'])['alert_count'].sum().reset_index()

# Visualize alert frequency by health state
fig = px.box(df, x='health_state', y='alert_count', 
             title='Alert Distribution by Engine Health State',
             color='health_state',
             color_discrete_map=colors)
fig.show()

# Alert frequency over cycles
fig2 = px.line(alert_analysis, x='cycle', y='alert_count', 
               color='health_state',
               title='Alert Count Evolution Over Engine Cycles',
               color_discrete_map=colors)
fig2.show()

print("🚨 Alert Analysis Summary:")
alert_summary = df.groupby('health_state')['alert_count'].agg(['mean', 'max', 'sum']).round(2)
display(alert_summary)

print("\n💡 Key Findings:")
print("• Critical engines generate significantly more alerts")
print("• Alert frequency increases with engine degradation")
print("• Healthy engines rarely trigger alerts")
print("• Alert patterns can be used for early warning systems")

## 8. Remaining Useful Life (RUL) Analysis

In [ ]:
# Analyze RUL patterns
fig = px.line(df, x='cycle', y='remaining_useful_life', 
              color='health_state',
              title='Remaining Useful Life by Engine Health State',
              color_discrete_map=colors)
fig.update_layout(xaxis_title='Engine Cycle', yaxis_title='Remaining Useful Life')
fig.show()

# RUL vs Key Sensor Relationships
fig2 = make_subplots(
    rows=1, cols=2,
    subplot_titles=['RUL vs Temperature T30', 'RUL vs Bypass Ratio']
)

# Temperature vs RUL
for health_state in ['healthy', 'degrading', 'critical']:
    data = df[df['health_state'] == health_state]
    fig2.add_trace(
        go.Scatter(
            x=data['sensor_3'], y=data['remaining_useful_life'],
            mode='markers',
            name=f'{health_state.capitalize()}',
            marker=dict(color=colors[health_state]),
            showlegend=True
        ),
        row=1, col=1
    )

# Bypass Ratio vs RUL
for health_state in ['healthy', 'degrading', 'critical']:
    data = df[df['health_state'] == health_state]
    fig2.add_trace(
        go.Scatter(
            x=data['sensor_11'], y=data['remaining_useful_life'],
            mode='markers',
            name=f'{health_state.capitalize()}',
            marker=dict(color=colors[health_state]),
            showlegend=False
        ),
        row=1, col=2
    )

fig2.update_layout(title="Sensor Values vs Remaining Useful Life", height=400)
fig2.show()

print("📈 RUL Analysis Summary:")
rul_stats = df.groupby('health_state')['remaining_useful_life'].agg(['mean', 'min', 'max']).round(0)
display(rul_stats)

print("\n🎯 RUL Insights:")
print("• Clear correlation between sensor values and remaining useful life")
print("• Temperature increases correspond to lower RUL")
print("• Bypass ratio shows strong relationship with engine degradation")
print("• RUL prediction models can leverage these sensor patterns")

## 9. Next Steps and Recommendations

Based on this data exploration, here are the recommended next steps:

### 🚀 **Immediate Actions**
1. **Set up real-time streaming** with Kafka to capture live sensor data
2. **Create monitoring dashboard** with key sensor trends and alerts
3. **Implement threshold-based alerting** for critical sensors

### 🤖 **Machine Learning Development**
1. **RUL Prediction Model**: Use temperature and bypass ratio as key features
2. **Anomaly Detection**: Implement multivariate analysis for early warning
3. **Health Classification**: Build models to automatically classify engine health states

### 📊 **Analytics Enhancement**
1. **Pattern Recognition**: Identify failure mode signatures
2. **Predictive Maintenance**: Optimize maintenance scheduling
3. **Fleet Analysis**: Compare engines and identify risk patterns

### 🏗️ **Infrastructure**
1. **Database Optimization**: Implement time-series optimizations
2. **API Development**: Create endpoints for external system integration
3. **Scaling**: Prepare for high-throughput production scenarios

## 10. Save Analysis Results

In [ ]:
# Save the simulated data for future analysis
output_dir = '../data/analysis'
os.makedirs(output_dir, exist_ok=True)

# Save datasets
df.to_csv(f'{output_dir}/simulated_engine_data.csv', index=False)
sensor_df.to_csv(f'{output_dir}/sensor_configuration.csv', index=False)
health_stats.to_csv(f'{output_dir}/health_state_statistics.csv')

print("💾 Analysis results saved to:")
print(f"  • {output_dir}/simulated_engine_data.csv")
print(f"  • {output_dir}/sensor_configuration.csv")
print(f"  • {output_dir}/health_state_statistics.csv")

print("\n✅ Data exploration completed successfully!")
print("🚀 Ready to proceed with real-time streaming and dashboard development.")